In [1]:
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import seaborn as sns
import celloracle as co
co.__version__

which: no R in (/home/msai/riemerpi001/.conda/envs/my_cell_env/bin:/cm/local/apps/environment-modules/4.5.3/bin:/home/msai/riemerpi001/.vscode-server/cli/servers/Stable-e170252f762678dec6ca2cc69aba1570769a5d39/server/bin/remote-cli:/cm/local/apps/environment-modules/4.5.3/bin:/home/msai/riemerpi001/.conda/envs/my_cell_env/bin:/apps/anaconda3/condabin:/cm/local/apps/gcc/11.2.0/bin:/home/msai/riemerpi001/.local/bin:/home/msai/riemerpi001/bin:/cm/local/apps/environment-modules/4.5.3/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/home/msai/riemerpi001/bin:/usr/bin:/usr/sbin:/cm/shared/apps/slurm/current/sbin:/tc2share/tc-scripts:/usr/share/centrifydc/bin:.:/sbin:/usr/sbin:/cm/local/apps/environment-modules/4.5.3/bin:/opt/dell/srvadmin/bin:/home/msai/riemerpi001/bin:/usr/bin:/usr/sbin:/cm/shared/apps/slurm/current/sbin:/tc2share/tc-scripts:/usr/share/centrifydc/bin:.)
2024-05-08 11:43:52,289 - INFO - Creating new config.
2024-05-08 11:43:52,292 - INFO - Using included version of AMD

'0.18.0'

In [2]:
# visualization settings
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

plt.rcParams['figure.figsize'] = [6, 4.5]
plt.rcParams["savefig.dpi"] = 300
save_folder = "figures"
os.makedirs(save_folder, exist_ok=True)

In [3]:
# Here, we will use a hematopoiesis data by Paul 2015.
# You can load preprocessed data using a celloracle function as follows.
adata = co.data.load_Paul2015_data()
adata

# Attention: Replace the data path below when using your data.
# adata = sc.read_h5ad("DATAPATH")

Data not found in the local folder. Loading data from github. Data will be saved at /home/msai/riemerpi001/celloracle_data/anndata


  0%|          | 0.00/63.2M [00:00<?, ?B/s]

AnnData object with n_obs × n_vars = 2671 × 1999
    obs: 'paul15_clusters', 'n_counts_all', 'n_counts', 'louvain', 'cell_type', 'louvain_annot', 'dpt_pseudotime'
    var: 'n_counts'
    uns: 'cell_type_colors', 'diffmap_evals', 'draw_graph', 'iroot', 'louvain', 'louvain_annot_colors', 'louvain_colors', 'louvain_sizes', 'neighbors', 'paga', 'paul15_clusters_colors', 'pca'
    obsm: 'X_diffmap', 'X_draw_graph_fa', 'X_pca'
    varm: 'PCs'
    layers: 'raw_count'
    obsp: 'connectivities', 'distances'

In [ ]:
print(f"Cell number is :{adata.shape[0]}")
print(f"Gene number is :{adata.shape[1]}")

# Random downsampling into 30K cells if the anndata object include more than 30 K cells.
n_cells_downsample = 30000
if adata.shape[0] > n_cells_downsample:
    # Let's dowmsample into 30K cells
    sc.pp.subsample(adata, n_obs=n_cells_downsample, random_state=123)
    
print(f"Cell number is :{adata.shape[0]}")


make an oracle object (from anndata)

In [ ]:
oracle = co.Oracle()
# In this notebook, we use the unscaled mRNA count for the nput of Oracle object.
adata.X = adata.layers["raw_count"].copy()

# Instantiate Oracle object.
oracle.import_anndata_as_raw_count(adata=adata,
                                   cluster_column_name="louvain_annot",
                                   embedding_name="X_draw_graph_fa")

manually add TF-target gene pair data. <- is htis something that I need to get more off? 

In [ ]:
!wget https://raw.githubusercontent.com/morris-lab/CellOracle/master/docs/demo_data/TF_data_in_Paul15.csv -O TF_data_in_Paul15.csv
Paul_15_data = pd.read_csv("TF_data_in_Paul15.csv")
Paul_15_data

make into python dict

In [ ]:
# Make dictionary: dictionary key is TF and dictionary value is list of target genes.
TF_to_TG_dictionary = {}

for TF, TGs in zip(Paul_15_data.TF, Paul_15_data.Target_genes):
    # convert target gene to list
    TG_list = TGs.replace(" ", "").split(",")
    # store target gene list in a dictionary
    TF_to_TG_dictionary[TF] = TG_list

# We invert the dictionary above using a utility function in celloracle.
TG_to_TF_dictionary = co.utility.inverse_dictionary(TF_to_TG_dictionary)

In [ ]:
oracle.addTFinfo_dictionary(TG_to_TF_dictionary)